# Feels Protocol Baseline Analysis

**Analysis Type:** Baseline Performance Assessment  
**Duration:** 2 weeks (336 hours)  
**Purpose:** Establish reference metrics and validate default parameters

## Overview

This notebook provides comprehensive analysis of the Feels protocol using current default parameters. It establishes baseline performance metrics for comparison with parameter optimization scenarios and validates the core simulation capabilities.

## Objectives

- **Simulate** baseline scenario with current protocol defaults (98.5% buffer, 1% treasury, 0.5% creator)
- **Analyze** floor price dynamics and POMM deployment patterns  
- **Examine** fee distribution and accumulation mechanisms
- **Visualize** key performance metrics and trading patterns
- **Compare** alternative fee scenarios to understand parameter sensitivity

## Key Metrics

- Floor price growth rate and market ratio trends
- POMM deployment frequency and efficiency
- Fee collection and distribution patterns
- Trading volume and liquidity utilization
- Protocol sustainability indicators

In [ ]:
# Import Libraries
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from datetime import datetime

# Configure matplotlib for inline plotting
%matplotlib inline
plt.rcParams['figure.facecolor'] = 'white'

# Package Import Configuration
try:
    # Production: Import from installed package
    from feels_sim.config import SimulationConfig
    from feels_sim.core import FeelsSimulation
    from feels_sim.metrics import (
        calculate_key_metrics, 
        calculate_hourly_aggregates,
        calculate_floor_to_market_ratio_stats,
        calculate_pomm_efficiency_metrics,
        snapshots_to_dataframe
    )
    from feels_sim.plotting import (
        setup_plot_style, create_summary_plots, create_detailed_analysis_plots,
        PlotStyle, DEFAULT_STYLE, plot_price_evolution, plot_volume_analysis,
        create_figure_with_style, save_plot
    )
    print("✓ Production package imports successful")
    
except ImportError:
    # Development: Import from local source
    print("→ Using local development setup...")
    project_root = Path().cwd().parent
    sys.path.insert(0, str(project_root))
    
    try:
        from feels_sim.config import SimulationConfig
        from feels_sim.core import FeelsSimulation
        from feels_sim.metrics import (
            calculate_key_metrics, 
            calculate_hourly_aggregates,
            calculate_floor_to_market_ratio_stats,
            calculate_pomm_efficiency_metrics,
            snapshots_to_dataframe
        )
        from feels_sim.plotting import (
            setup_plot_style, create_summary_plots, create_detailed_analysis_plots,
            PlotStyle, DEFAULT_STYLE, plot_price_evolution, plot_volume_analysis,
            create_figure_with_style, save_plot
        )
        print("✓ Development imports successful")
    except ImportError as e:
        print(f"✗ Import failed: {e}")
        raise

# Initialize Plotting
setup_plot_style()
print("✓ Plotting configuration complete")

## Setup and Imports

## 1. Baseline Simulation

Run the default protocol configuration and capture comprehensive performance metrics.

In [ ]:
# Run baseline simulation with current protocol defaults
print("Running baseline simulation (336 hours)...")

config = SimulationConfig()  # Uses current defaults: protocol=100bps, creator=50bps, buffer gets remainder
print(f"Fee split: protocol={config.protocol_fee_rate_bps}bps, creator={config.creator_fee_rate_bps}bps, buffer gets remainder")

sim = FeelsSimulation(config)
baseline_results = sim.run(hours=336)  # 2 weeks

print(f"Simulation completed: {len(baseline_results.snapshots)} snapshots, {len(baseline_results.hourly_aggregates)} hourly aggregates")

In [ ]:
# Analyze baseline results
baseline_analysis = calculate_key_metrics(baseline_results.snapshots)

print("=== BASELINE SIMULATION RESULTS ===")
print(f"Simulation Duration: {baseline_analysis['simulation_hours']:.1f} hours")
print(f"Initial SOL Price: ${baseline_analysis['initial_sol_price']:.2f}")
print(f"Final SOL Price: ${baseline_analysis['final_sol_price']:.2f}")
print(f"Initial Floor Price: ${baseline_analysis['initial_floor_price']:.6f}")
print(f"Final Floor Price: ${baseline_analysis['final_floor_price']:.6f}")
print(f"")
print(f"Floor Growth Rate (Annual): {baseline_analysis['floor_growth_rate_annual']:.2%}")
print(f"Average Floor/Market Ratio: {baseline_analysis['avg_floor_to_market_ratio']:.2%}")
print(f"Total POMM Deployments: {baseline_analysis['pomm_deployments']}")
print(f"Total Trading Volume: {baseline_analysis['total_volume']:,.0f} FeelsSOL")
print(f"Total Fees Collected: {baseline_analysis['total_fees']:,.2f} FeelsSOL")
print(f"Protocol Efficiency: {baseline_analysis['protocol_efficiency']:.6f} USD/FeelsSOL")
print(f"")
print(f"Final Treasury Balance: {baseline_analysis['final_treasury_balance']:,.2f} FeelsSOL")
print(f"Final Buffer Balance: {baseline_analysis['final_buffer_balance']:,.2f} FeelsSOL")
print(f"Final Deployed FeelsSOL: {baseline_analysis['final_deployed_feelssol']:,.2f} FeelsSOL")
print(f"Buffer Routed (Cumulative): {baseline_analysis['buffer_routed_cumulative']:,.2f} FeelsSOL")
print(f"Mint Cumulative: {baseline_analysis['mint_cumulative']:,.2f} FeelsSOL")
print(f"LP Fees Distributed: {baseline_analysis.get('lp_fees_total', 0):,.2f} FeelsSOL")

## 2. Performance Visualization

Generate comprehensive charts and analysis plots for baseline performance assessment.

In [ ]:
# Create comprehensive visualizations using centralized plotting functions
print("Creating summary plots...")
create_summary_plots(baseline_results)
plt.show()

print("\nCreating detailed analysis plots...")
create_detailed_analysis_plots(baseline_results)
plt.show()

In [ ]:
# Create additional analysis tables and visualizations
df = snapshots_to_dataframe(baseline_results.snapshots)

if not df.is_empty():
    # Price evolution table - more precise than chart
    print("=== PRICE EVOLUTION TABLE ===")
    
    # Convert to pandas DataFrame
    df_pandas = df.to_pandas()
    
    # Create daily summary table (every 24 hours)
    daily_intervals = list(range(0, len(df_pandas), 24))  # Every 24 hours
    
    print(f"{'Day':<4} {'Hour':<5} {'Floor Price':<12} {'SOL Price':<11} {'Floor/Market':<12} {'POMM Deps':<10} {'Volume':<10}")
    print("-" * 70)
    
    for i, hour_idx in enumerate(daily_intervals):
        if hour_idx < len(df_pandas):
            row = df_pandas.iloc[hour_idx]
            day = hour_idx // 24
            hour = hour_idx
            floor_price = row['floor_price_usd']
            sol_price = row['sol_price_usd']
            ratio_pct = (floor_price / sol_price) * 100
            
            # Count POMM deployments in this day (next 24 hours)
            end_idx = min(hour_idx + 24, len(df_pandas))
            day_data = df_pandas.iloc[hour_idx:end_idx]
            pomm_deps = len(day_data[day_data['pomm_deployment'] == True]) if 'pomm_deployment' in day_data.columns else 0
            
            # Sum volume for this day
            day_volume = day_data['volume_feelssol'].sum() if 'volume_feelssol' in day_data.columns else 0
            
            print(f"{day:<4} {hour:<5} ${floor_price:<11.6f} ${sol_price:<10.2f} {ratio_pct:<11.3f}% {pomm_deps:<10} {day_volume:<10.0f}")
    
    print()
    print("Key Metrics Summary:")
    print(f"Initial Floor Price: ${df_pandas.iloc[0]['floor_price_usd']:.6f}")
    print(f"Final Floor Price:   ${df_pandas.iloc[-1]['floor_price_usd']:.6f}")
    print(f"Price Increase:      ${df_pandas.iloc[-1]['floor_price_usd'] - df_pandas.iloc[0]['floor_price_usd']:.6f}")
    print(f"Relative Growth:     {((df_pandas.iloc[-1]['floor_price_usd'] / df_pandas.iloc[0]['floor_price_usd']) - 1) * 100:.3f}%")
    print()
    
    # Volume analysis plot  
    print("Creating volume analysis...")
    fig, ax = plot_volume_analysis(df.to_pandas())
    if fig:
        # Update x-axis to show days instead of hours
        ax.set_xlabel('Days')
        # Set x-axis limits with tiny margins (336 hours = 14 days)
        ax.set_xlim(-8, 344)  # Add ~0.33 day margin on each side
        # Set ticks at round day intervals (every 24 hours = 1 day)
        day_ticks = list(range(0, 337, 48))  # Every 2 days in hours
        ax.set_xticks(day_ticks)
        ax.set_xticklabels([f'{tick/24:.0f}' for tick in day_ticks])
        plt.show()
        
    # Custom 3-hour analysis with centralized styling - better readability than hourly
    print("Creating 3-hour analysis...")
    
    # Import the time aggregates function
    from feels_sim.metrics import calculate_time_aggregates
    
    # Use 3-hour bins for better visualization of 14-day simulation
    three_hour_data = calculate_time_aggregates(baseline_results.snapshots, "3h")
    
    if three_hour_data:
        fig, axes = create_figure_with_style(2, 2, DEFAULT_STYLE.figure_size_detailed)
        if fig:
            # Extract data for plotting - convert 3-hour intervals to days
            intervals = list(range(len(three_hour_data)))
            days = [i * 3 / 24 for i in intervals]  # Convert 3-hour intervals to days
            volumes = [h['volume_feelssol'] for h in three_hour_data]
            fees = [h['fees_collected'] for h in three_hour_data]
            floor_prices = [h['avg_floor_price'] for h in three_hour_data]
            deployments = [h['pomm_deployments'] for h in three_hour_data]
            
            # Calculate appropriate bar width for 3-hour bins
            bar_width = 0.8 * 3 / 24  # Width proportional to 3-hour intervals in days
            
            # Set consistent x-axis limits and ticks for all subplots
            max_days = max(days) if days else 14
            margin = 0.3  # Small margin in days
            day_tick_interval = 2  # Every 2 days
            day_ticks = [i for i in range(0, int(max_days) + 1, day_tick_interval)]
            
            # Trading volume - Fixed title
            axes[0, 0].bar(days, volumes, width=bar_width, alpha=DEFAULT_STYLE.alpha, 
                          edgecolor=DEFAULT_STYLE.edgecolor, linewidth=0.5)
            axes[0, 0].set_title('Trading Volume (3-hour bins)')
            axes[0, 0].set_ylabel('FeelsSOL')
            axes[0, 0].set_xlabel('Days')
            axes[0, 0].set_xlim(-margin, max_days + margin)
            axes[0, 0].set_xticks(day_ticks)
            
            # Fees collected - Fixed title
            axes[0, 1].bar(days, fees, width=bar_width, alpha=DEFAULT_STYLE.alpha,
                          edgecolor=DEFAULT_STYLE.edgecolor, linewidth=0.5)
            axes[0, 1].set_title('Fees Collected (3-hour bins)')
            axes[0, 1].set_ylabel('FeelsSOL')
            axes[0, 1].set_xlabel('Days')
            axes[0, 1].set_xlim(-margin, max_days + margin)
            axes[0, 1].set_xticks(day_ticks)
            
            # Floor price trend - Fixed title
            axes[1, 0].plot(days, floor_prices, marker='o', linewidth=DEFAULT_STYLE.linewidth,
                           markersize=6, markerfacecolor='white', 
                           markeredgewidth=DEFAULT_STYLE.marker_edgewidth)
            axes[1, 0].set_title('Average Floor Price (3-hour bins)')
            axes[1, 0].set_ylabel('USD')
            axes[1, 0].set_xlabel('Days')
            axes[1, 0].set_xlim(-margin, max_days + margin)
            axes[1, 0].set_xticks(day_ticks)
            
            # POMM deployment frequency - Fixed title
            axes[1, 1].bar(days, deployments, width=bar_width, alpha=DEFAULT_STYLE.alpha,
                          edgecolor=DEFAULT_STYLE.edgecolor, linewidth=0.5)
            axes[1, 1].set_title('POMM Deployments (3-hour bins)')
            axes[1, 1].set_ylabel('Count')
            axes[1, 1].set_xlabel('Days')
            axes[1, 1].set_xlim(-margin, max_days + margin)
            axes[1, 1].set_xticks(day_ticks)
            
            plt.suptitle('3-Hour Binned Analysis - Baseline Simulation')
            plt.tight_layout()
            plt.show()
else:
    print("No data available for plotting")

## 3. Statistical Analysis

Calculate detailed statistical metrics for protocol performance assessment.

In [ ]:
# Floor/market ratio statistics
ratio_stats = calculate_floor_to_market_ratio_stats(baseline_results.snapshots)

print("=== FLOOR/MARKET RATIO STATISTICS ===")
for stat, value in ratio_stats.items():
    print(f"{stat}: {value:.4f}")

In [ ]:
# POMM efficiency metrics
pomm_metrics = calculate_pomm_efficiency_metrics(baseline_results.snapshots)

print("=== POMM DEPLOYMENT EFFICIENCY ===")
for metric, value in pomm_metrics.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.2f}")
    else:
        print(f"{metric}: {value}")

## 4. Fee Distribution Analysis

Examine how fees are allocated across protocol components and validate against expected distributions.

In [ ]:
# Analyze fee distribution patterns
total_fees = baseline_analysis['total_fees']
treasury_fees = baseline_analysis['final_treasury_balance']
buffer_fees = baseline_analysis['buffer_routed_cumulative']

# Calculate creator fees from final state
creator_fees = baseline_results.snapshots[-1].floor_state.creator_balance

print("=== FEE DISTRIBUTION ANALYSIS ===")
print(f"Total Fees Collected: {total_fees:,.2f} FeelsSOL")
print(f"")
print(f"Buffer Allocation: {buffer_fees:,.2f} FeelsSOL ({buffer_fees/total_fees*100:.1f}%)")
print(f"Treasury Allocation: {treasury_fees:,.2f} FeelsSOL ({treasury_fees/total_fees*100:.1f}%)")
print(f"Creator Allocation: {creator_fees:,.2f} FeelsSOL ({creator_fees/total_fees*100:.1f}%)")
print(f"")
print(f"Expected vs Actual Distribution:")
# Calculate expected percentages from basis points
buffer_share_pct = 100.0 - (config.protocol_fee_rate_bps + config.creator_fee_rate_bps) / 100.0
treasury_share_pct = config.protocol_fee_rate_bps / 100.0
creator_share_pct = config.creator_fee_rate_bps / 100.0

print(f"Buffer: Expected {buffer_share_pct:.1f}%, Actual {buffer_fees/total_fees*100:.1f}%")
print(f"Treasury: Expected {treasury_share_pct:.1f}%, Actual {treasury_fees/total_fees*100:.1f}%")
print(f"Creator: Expected {creator_share_pct:.1f}%, Actual {creator_fees/total_fees*100:.1f}%")

## 5. Scenario Comparison

Compare baseline performance against alternative fee scenarios to understand parameter sensitivity.

In [ ]:
# Compare different fee scenarios
scenarios = [
    ("default", "Current Default"),
    ("protocol_sustainable", "Protocol Sustainable"), 
    ("creator_incentive", "Creator Incentive"),
    ("balanced_growth", "Balanced Growth")
]

scenario_results = {}

print("Running scenario comparison analysis...")

for scenario_key, scenario_name in scenarios:
    print(f"Running {scenario_name}...")
    
    config = SimulationConfig.create_fee_scenario(scenario_key)
    sim = FeelsSimulation(config)
    results = sim.run(hours=336)
    analysis = calculate_key_metrics(results.snapshots)
    
    # Calculate fee percentages from basis points
    buffer_pct = 100.0 - (config.protocol_fee_rate_bps + config.creator_fee_rate_bps) / 100.0
    treasury_pct = config.protocol_fee_rate_bps / 100.0
    creator_pct = config.creator_fee_rate_bps / 100.0
    
    scenario_results[scenario_key] = {
        "name": scenario_name,
        "config": {
            "buffer": buffer_pct,
            "treasury": treasury_pct,
            "creator": creator_pct,
            "protocol_fee_rate_bps": config.protocol_fee_rate_bps,
            "creator_fee_rate_bps": config.creator_fee_rate_bps
        },
        "analysis": analysis
    }
    
    print(f"  Floor growth: {analysis['floor_growth_rate_annual']:.2%}, "
          f"POMM deployments: {analysis['pomm_deployments']}, "
          f"Treasury: {analysis['final_treasury_balance']:.1f}")

print("Scenario comparison completed")

In [ ]:
# Create comparative visualization using centralized styling
fig, axes = create_figure_with_style(2, 2, DEFAULT_STYLE.figure_size_detailed)
if fig:
    fig.suptitle('Fee Scenario Comparison', fontsize=16)

    scenarios_list = list(scenario_results.keys())
    scenario_names = [scenario_results[s]["name"] for s in scenarios_list]
    colors = DEFAULT_STYLE.color_palette

    # Floor growth rates
    floor_growth = [scenario_results[s]["analysis"]["floor_growth_rate_annual"] for s in scenarios_list]
    axes[0, 0].bar(range(len(scenarios_list)), [x*100 for x in floor_growth], 
                   color=colors[0], alpha=DEFAULT_STYLE.alpha, 
                   edgecolor=DEFAULT_STYLE.edgecolor, linewidth=1)
    axes[0, 0].set_title('Floor Growth Rate (Annual %)')
    axes[0, 0].set_xticks(range(len(scenarios_list)))
    axes[0, 0].set_xticklabels([s.replace('_', '\n') for s in scenarios_list], rotation=45)

    # Treasury balances
    treasury_balances = [scenario_results[s]["analysis"]["final_treasury_balance"] for s in scenarios_list]
    axes[0, 1].bar(range(len(scenarios_list)), treasury_balances,
                   color=colors[1], alpha=DEFAULT_STYLE.alpha,
                   edgecolor=DEFAULT_STYLE.edgecolor, linewidth=1)
    axes[0, 1].set_title('Final Treasury Balance (FeelsSOL)')
    axes[0, 1].set_xticks(range(len(scenarios_list)))
    axes[0, 1].set_xticklabels([s.replace('_', '\n') for s in scenarios_list], rotation=45)

    # POMM deployments
    pomm_deployments = [scenario_results[s]["analysis"]["pomm_deployments"] for s in scenarios_list]
    axes[1, 0].bar(range(len(scenarios_list)), pomm_deployments,
                   color=colors[2], alpha=DEFAULT_STYLE.alpha,
                   edgecolor=DEFAULT_STYLE.edgecolor, linewidth=1)
    axes[1, 0].set_title('Total POMM Deployments')
    axes[1, 0].set_xticks(range(len(scenarios_list)))
    axes[1, 0].set_xticklabels([s.replace('_', '\n') for s in scenarios_list], rotation=45)

    # Protocol efficiency
    efficiency = [scenario_results[s]["analysis"]["protocol_efficiency"] for s in scenarios_list]
    axes[1, 1].bar(range(len(scenarios_list)), efficiency,
                   color=colors[3], alpha=DEFAULT_STYLE.alpha,
                   edgecolor=DEFAULT_STYLE.edgecolor, linewidth=1)
    axes[1, 1].set_title('Protocol Efficiency (USD/FeelsSOL)')
    axes[1, 1].set_xticks(range(len(scenarios_list)))
    axes[1, 1].set_xticklabels([s.replace('_', '\n') for s in scenarios_list], rotation=45)

    plt.tight_layout()
    plt.show()
else:
    print("Could not create comparison plots")

## 6. Key Insights

Generate actionable insights and recommendations based on baseline analysis results.

In [ ]:
# Generate key insights
print("=== KEY INSIGHTS FROM BASELINE ANALYSIS ===")
print()

# Best performing scenario analysis
best_floor_growth = max(scenario_results.items(), key=lambda x: x[1]["analysis"]["floor_growth_rate_annual"])
best_treasury = max(scenario_results.items(), key=lambda x: x[1]["analysis"]["final_treasury_balance"])
best_efficiency = max(scenario_results.items(), key=lambda x: x[1]["analysis"]["protocol_efficiency"])

print(f"1. Floor Growth Performance:")
print(f"   Best: {best_floor_growth[1]['name']} ({best_floor_growth[1]['analysis']['floor_growth_rate_annual']:.2%} annual)")
print(f"   Current default achieves {baseline_analysis['floor_growth_rate_annual']:.2%} annual growth")
print()

print(f"2. Treasury Accumulation:")
print(f"   Best: {best_treasury[1]['name']} ({best_treasury[1]['analysis']['final_treasury_balance']:.1f} FeelsSOL)")
print(f"   Current default accumulates {baseline_analysis['final_treasury_balance']:.1f} FeelsSOL")
print()

print(f"3. Protocol Efficiency:")
print(f"   Best: {best_efficiency[1]['name']} ({best_efficiency[1]['analysis']['protocol_efficiency']:.6f} USD/FeelsSOL)")
print(f"   Current default achieves {baseline_analysis['protocol_efficiency']:.6f} USD/FeelsSOL efficiency")
print()

print(f"4. Trade-offs Analysis:")
current_config = scenario_results['default']['config']
sustainable_config = scenario_results['protocol_sustainable']['config']

treasury_diff = best_treasury[1]['analysis']['final_treasury_balance'] - baseline_analysis['final_treasury_balance']
floor_diff = best_floor_growth[1]['analysis']['floor_growth_rate_annual'] - baseline_analysis['floor_growth_rate_annual']

print(f"   Moving from current default to protocol sustainable:")
print(f"   - Treasury gains: +{treasury_diff:.1f} FeelsSOL")
print(f"   - Floor growth change: {floor_diff:+.2%}")
print(f"   - Buffer allocation change: {sustainable_config['buffer'] - current_config['buffer']:+.1f}%")
print()

print(f"5. POMM Deployment Patterns:")
print(f"   Baseline deployments: {baseline_analysis['pomm_deployments']} over {baseline_analysis['simulation_hours']:.0f} hours")
print(f"   Deployment frequency: {baseline_analysis['pomm_deployments'] / baseline_analysis['simulation_hours'] * 24:.1f} per day")
if baseline_analysis['pomm_deployments'] > 0:
    print(f"   Average deployment drives floor up by: {(baseline_analysis['final_floor_price'] - baseline_analysis['initial_floor_price']) / baseline_analysis['pomm_deployments']:.8f} USD")

## 7. Data Export

Export baseline results for use in parameter sweep and comparative analysis.

In [ ]:
# Export results for use in parameter sweep notebook
baseline_export = {
    "timestamp": datetime.now().isoformat(),
    "baseline_config": {
        "protocol_fee_rate_bps": config.protocol_fee_rate_bps,
        "creator_fee_rate_bps": config.creator_fee_rate_bps,
        "base_fee_bps": config.base_fee_bps,
    },
    "baseline_analysis": baseline_analysis,
    "scenario_comparison": scenario_results,
    "ratio_statistics": ratio_stats,
    "pomm_metrics": pomm_metrics
}

# Setup export path for both local and online execution
try:
    # Try to use project_root if it was set in the import section
    export_dir = project_root / "experiments" / "runs"
except NameError:
    # If running from installed package, use current directory
    export_dir = Path("experiments") / "runs"

export_dir.mkdir(parents=True, exist_ok=True)
export_path = export_dir / "baseline_analysis.json"

with open(export_path, 'w') as f:
    json.dump(baseline_export, f, indent=2)

print(f"Baseline analysis results exported to {export_path}")
print("Use this data in parameter sweep analysis (02_parameter_sweep.ipynb)")

In [ ]:
from IPython.display import Markdown

floor_bps = (baseline_analysis['final_floor_price'] / baseline_analysis['initial_floor_price'] - 1.0) * 10_000
run_days = baseline_analysis['simulation_hours'] / 24
pomm_per_day = baseline_analysis['pomm_deployments'] / run_days if run_days else 0
lp_fees = baseline_analysis.get('lp_fees_total')
scenario_growth = {key: value['analysis']['floor_growth_rate_annual'] * 10_000 for key, value in scenario_results.items()}
scenario_treasury = {key: value['analysis']['final_treasury_balance'] for key, value in scenario_results.items()}

best_growth_key = max(scenario_growth, key=scenario_growth.get)
best_treasury_key = max(scenario_treasury, key=scenario_treasury.get)

summary_lines = [
    f"Floor advanced from ${baseline_analysis['initial_floor_price']:.6f} to ${baseline_analysis['final_floor_price']:.6f} (+{floor_bps:.1f} bps) over {run_days:.1f} days.",
    f"Trading volume totalled {baseline_analysis['total_volume']:,.0f} FeelsSOL, producing {baseline_analysis['total_fees']:,.2f} fees with {baseline_analysis['final_treasury_balance']:,.2f} FeelsSOL accruing to treasury and {baseline_analysis['final_buffer_balance']:,.2f} FeelsSOL remaining in buffer.",
]
if lp_fees is not None:
    summary_lines.append(f"LPs accumulated {lp_fees:,.2f} FeelsSOL in protocol fees during the run.")
summary_lines.append(f"POMM deployed {baseline_analysis['pomm_deployments']} times (~{pomm_per_day:.1f} per day).")
summary_lines.append(
    f"Among compared scenarios, {scenario_results[best_growth_key]['name']} delivered the fastest annualised floor growth (~{scenario_growth[best_growth_key]:.2f} bps), while {scenario_results[best_treasury_key]['name']} accumulated the largest treasury balance ({scenario_treasury[best_treasury_key]:,.2f} FeelsSOL)."
)

Markdown("## Summary\n\n" + "\n".join(f"- {line}" for line in summary_lines))